## 학습목표
 1. DataFrame groupby 이해하기

In [1]:
import pandas as pd
import numpy as np

#### group by
  + 아래의 세 단계를 적용하여 데이터를 그룹화(groupping) (SQL의 group by 와 개념적으로는 동일, 사용법은 유사)
    - 데이터 분할
    - operation 적용
    - 데이터 병합

In [2]:
# data 출처: https://www.kaggle.com/hesh97/titanicdataset-traincsv/data
df = pd.read_csv('./train.csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


#### GroupBy groups 속성
 - 각 그룹과 그룹에 속한 index를 dict 형태로 표현

In [3]:
class_group = df.groupby('Pclass')
class_group

In [4]:
class_group.groups

{1: Int64Index([  1,   3,   6,  11,  23,  27,  30,  31,  34,  35,
             ...
             853, 856, 857, 862, 867, 871, 872, 879, 887, 889],
            dtype='int64', length=216),
 2: Int64Index([  9,  15,  17,  20,  21,  33,  41,  43,  53,  56,
             ...
             848, 854, 861, 864, 865, 866, 874, 880, 883, 886],
            dtype='int64', length=184),
 3: Int64Index([  0,   2,   4,   5,   7,   8,  10,  12,  13,  14,
             ...
             875, 876, 877, 878, 881, 882, 884, 885, 888, 890],
            dtype='int64', length=491)}

In [5]:
gender_group = df.groupby('Sex')
gender_group.groups

{'female': Int64Index([  1,   2,   3,   8,   9,  10,  11,  14,  15,  18,
             ...
             866, 871, 874, 875, 879, 880, 882, 885, 887, 888],
            dtype='int64', length=314),
 'male': Int64Index([  0,   4,   5,   6,   7,  12,  13,  16,  17,  20,
             ...
             873, 876, 877, 878, 881, 883, 884, 886, 889, 890],
            dtype='int64', length=577)}

#### groupping 함수
 - 그룹 데이터에 적용 가능한 통계 함수(NaN은 제외하여 연산)
 - count - 데이터 개수 
 - sum   - 데이터의 합
 - mean, std, var - 평균, 표준편차, 분산
 - min, max - 최소, 최대값

In [6]:
class_group.count()

,PassengerId,Survived,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
Pclass,,,,,,,,,,,
1,216,216,216,216,186,216,216,216,216,176,214
2,184,184,184,184,173,184,184,184,184,16,184
3,491,491,491,491,355,491,491,491,491,12,491


In [7]:
class_group.mean()['Survived']

Pclass
1    0.629630
2    0.472826
3    0.242363
Name: Survived, dtype: float64

In [8]:
class_group.max()

,PassengerId,Survived,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
Pclass,,,,,,,,,,,
1,890,1,"Young, Miss. Marie Grice",male,80.0,3,4,WE/P 5735,512.3292,T,S
2,887,1,"del Carlo, Mr. Sebastiano",male,70.0,3,3,W/C 14208,73.5000,F4,S
3,891,1,"van Melkebeke, Mr. Philemon",male,74.0,8,6,W./C. 6609,69.5500,G6,S


In [9]:
class_group.max().index

Int64Index([1, 2, 3], dtype='int64', name=u'Pclass')

* 성별에 따른 생존율 구해보기

In [10]:
df.groupby('Sex').mean()['Survived']

Sex
female    0.742038
male      0.188908
Name: Survived, dtype: float64

#### 복수 columns로 groupping 하기
 - groupby에 column 리스트를 전달
 - 통계함수를 적용한 결과는 multiindex를 갖는 dataframe

* 클래스와 성별에 따른 생존률 구해보기

In [11]:
df.groupby(['Pclass', 'Sex']).mean()['Survived']

Pclass  Sex   
1       female    0.968085
        male      0.368852
2       female    0.921053
        male      0.157407
3       female    0.500000
        male      0.135447
Name: Survived, dtype: float64

In [12]:
df.groupby(['Pclass', 'Sex']).mean().loc[(2, 'female')]

PassengerId    443.105263
Survived         0.921053
Age             28.722973
SibSp            0.486842
Parch            0.605263
Fare            21.970121
Name: (2, female), dtype: float64

In [13]:
df.groupby(['Pclass', 'Sex']).mean().index

MultiIndex(levels=[[1, 2, 3], [u'female', u'male']],
           codes=[[0, 0, 1, 1, 2, 2], [0, 1, 0, 1, 0, 1]],
           names=[u'Pclass', u'Sex'])

#### index를 이용한 group by
 - index가 있는 경우, groupby 함수에 level 사용 가능
   - level은 index의 depth를 의미하며, 가장 왼쪽부터 0부터 증가

* **set_index** 함수
 - column 데이터를 index 레벨로 변경
* **reset_index** 함수
 - 인덱스 초기화

In [17]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [18]:
df.set_index(['Pclass', 'Sex']).reset_index()

,Pclass,Sex,PassengerId,Survived,Name,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,male,1,0,"Braund, Mr. Owen Harris",22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,female,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",38.0,1,0,PC 17599,71.2833,C85,C
2,3,female,3,1,"Heikkinen, Miss. Laina",26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,female,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",35.0,1,0,113803,53.1000,C123,S
4,3,male,5,0,"Allen, Mr. William Henry",35.0,0,0,373450,8.0500,NaN,S
5,3,male,6,0,"Moran, Mr. James",NaN,0,0,330877,8.4583,NaN,Q
6,1,male,7,0,"McCarthy, Mr. Timothy J",54.0,0,0,17463,51.8625,E46,S
7,3,male,8,0,"Palsson, Master. Gosta Leonard",2.0,3,1,349909,21.0750,NaN,S
8,3,female,9,1,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",27.0,0,2,347742,11.1333,NaN,S
9,2,female,10,1,"Nasser, Mrs. Nicholas (Adele Achem)",14.0,1,0,237736,30.0708,NaN,C


In [19]:
df.set_index('Age').groupby(level=0).mean()

,PassengerId,Survived,Pclass,SibSp,Parch,Fare
Age,,,,,,
0.42,804.000000,1.000000,3.000000,0.000000,1.000000,8.516700
0.67,756.000000,1.000000,2.000000,1.000000,1.000000,14.500000
0.75,557.500000,1.000000,3.000000,2.000000,1.000000,19.258300
0.83,455.500000,1.000000,2.000000,0.500000,1.500000,23.875000
0.92,306.000000,1.000000,1.000000,1.000000,2.000000,151.550000
1.00,415.428571,0.714286,2.714286,1.857143,1.571429,30.005957
2.00,346.900000,0.300000,2.600000,2.100000,1.300000,37.536250
3.00,272.000000,0.833333,2.500000,1.833333,1.333333,25.781950
4.00,466.100000,0.700000,2.600000,1.600000,1.400000,29.543330


#### 나이대별로 생존율 구하기

In [20]:
import math
def age_categorize(age):
    if math.isnan(age):
        return -1
    return math.floor(age / 10) * 10

In [21]:
df.set_index('Age').groupby(age_categorize).mean()['Survived']

-1.0     0.293785
 0.0     0.612903
 10.0    0.401961
 20.0    0.350000
 30.0    0.437126
 40.0    0.382022
 50.0    0.416667
 60.0    0.315789
 70.0    0.000000
 80.0    1.000000
Name: Survived, dtype: float64

#### MultiIndex를 이용한 groupping

In [22]:
df.set_index(['Pclass', 'Sex']).groupby(level=[0, 1]).mean()['Age']

Pclass  Sex   
1       female    34.611765
        male      41.281386
2       female    28.722973
        male      30.740707
3       female    21.750000
        male      26.507589
Name: Age, dtype: float64

#### aggregate(집계) 함수 사용하기
 - groupby 결과에 집계함수를 적용하여 그룹별 데이터 확인 가능

In [23]:
df.set_index(['Pclass', 'Sex']).groupby(level=[0, 1]).aggregate([np.mean, np.sum, np.max])

PassengerId               Survived                 Age           \
                     mean     sum amax      mean sum amax       mean      sum   
Pclass Sex                                                                      
1      female  469.212766   44106  888  0.968085  91    1  34.611765  2942.00   
       male    455.729508   55599  890  0.368852  45    1  41.281386  4169.42   
2      female  443.105263   33676  881  0.921053  70    1  28.722973  2125.50   
       male    447.962963   48380  887  0.157407  17    1  30.740707  3043.33   
3      female  399.729167   57561  889  0.500000  72    1  21.750000  2218.50   
       male    455.515850  158064  891  0.135447  47    1  26.507589  6706.42   

                        SibSp               Parch                  Fare  \
               amax      mean  sum amax      mean  sum amax        mean   
Pclass Sex                                                                
1      female  63.0  0.553191   52    3  0.457447   43    2  106.125798   
       male    80.0  0.311475   38    3  0.278689   34    4   67.226127   
2      female  57.0  0.486842   37    3  0.605263   46    3   21.970121   
       male    70.0  0.342593   37    2  0.222222   24    2   19.741782   
3      female  63.0  0.895833  129    8  0.798611  115    6   16.118810   
       male    74.0  0.498559  173    8  0.224784   78    5   12.661633   

                                    
                     sum      amax  
Pclass Sex                          
1      female  9975.8250  512.3292  
       male    8201.5875  512.3292  
2      female  1669.7292   65.0000  
       male    2132.1125   73.5000  
3      female  2321.1086   69.5500  
       male    4393.5865   69.5500